# VacationPy
---

## Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,moyynkum,44.2914,72.9469,15.96,81,87,14.56,KZ,1738728157
1,1,atafu village,-8.5421,-172.5159,83.44,70,15,12.53,TK,1738728157
2,2,port-aux-francais,-49.3500,70.2167,42.33,97,96,24.67,TF,1738728157
3,3,grytviken,-54.2811,-36.5092,36.63,88,69,18.12,GS,1738728157
4,4,port elizabeth,-33.9180,25.5701,71.31,99,94,11.99,ZA,1738727976


In [3]:
# Do a len to see how many cities are in the data
print(len(city_data_df))

553


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
mapPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    scale = 1,
    color= "City",
    alpha = 0.7,
    frame_height = 504,
    frame_width = 896,
    tiles="OSM",
    hover_cols = ["City"]
        
)

# Display the map
mapPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Convert Max Temp to Celsius (from Imperial to Metric)
city_data_df["Max Temp"] = (city_data_df["Max Temp"] - 32) * 5/9
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,moyynkum,44.2914,72.9469,-8.911111,81,87,14.56,KZ,1738728157
1,1,atafu village,-8.5421,-172.5159,28.577778,70,15,12.53,TK,1738728157
2,2,port-aux-francais,-49.3500,70.2167,5.738889,97,96,24.67,TF,1738728157
3,3,grytviken,-54.2811,-36.5092,2.572222,88,69,18.12,GS,1738728157
4,4,port elizabeth,-33.9180,25.5701,21.838889,99,94,11.99,ZA,1738727976


In [7]:
# Narrow down cities that fit criteria
idealCityDF = city_data_df.loc[
	(city_data_df["Max Temp"] < 27) & 
	(city_data_df["Max Temp"] > 21) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]
# Drop the null values
idealCityDF = idealCityDF.dropna()

idealCityDF.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,coahuayana de hidalgo,18.7000,-103.6583,21.827778,94,0,3.49,MX,1738728158
256,256,ixtapa,20.7000,-105.2000,21.788889,62,0,3.22,MX,1738728005
331,331,guanica,17.9716,-66.9080,24.361111,48,0,3.80,PR,1738728187
426,426,ovalle,-30.5983,-71.2003,22.388889,45,0,3.06,CL,1738728149


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealCityDF[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
hotel_df.head()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
16,coahuayana de hidalgo,MX,18.7000,-103.6583,94,
256,ixtapa,MX,20.7000,-105.2000,62,
331,guanica,PR,17.9716,-66.9080,48,
426,ovalle,CL,-30.5983,-71.2003,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel

# Set radius to 10000 M (10 KM)
radius = 10000
# set parameters that remain constant outside the below loop
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
coahuayana de hidalgo - nearest hotel: No hotel found
ixtapa - nearest hotel: Ma. Cristina
guanica - nearest hotel: Copamarina Beach Resort
ovalle - nearest hotel: Hotel Roxy


,City,Country,Lat,Lng,Humidity,Hotel Name
16,coahuayana de hidalgo,MX,18.7000,-103.6583,94,No hotel found
256,ixtapa,MX,20.7000,-105.2000,62,Ma. Cristina
331,guanica,PR,17.9716,-66.9080,48,Copamarina Beach Resort
426,ovalle,CL,-30.5983,-71.2003,45,Hotel Roxy


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the hotel map plot
hotelPlot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    scale = 2,
    color= "City",
    alpha = 0.7,
    frame_height = 600,
    frame_width = 900,
    tiles="OSM",
    hover_cols = ["Hotel Name", "Country"]
        
)
# Display the map
hotelPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)